In [46]:
%matplotlib widget
# %matplotlib inline
# %matplotlib qt 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
from shapely.geometry import shape, GeometryCollection
from pyproj import CRS
from tqdm.notebook import tqdm
from tqdm import trange

import ee
ee.Initialize()

import geemap.eefolium as emap
create_map = lambda: emap.Map()

In [3]:
df = pd.read_parquet('../data/out/icesat-2-DK.parquet')

## Query water occurrence values for all ICESat-2 points

In [10]:
image = ee.Image('users/gena/eo-bathymetry/NDWI/NDWI-4521-v9')

In [5]:
print(image.reduceRegion(reducer=ee.Reducer.minMax(), geometry=image.geometry(), scale=90).getInfo())

{'water_score_max': 0.15000000000000024, 'water_score_min': -0.10000000000000006}


In [30]:
df.columns

Index(['track_id', 'date', 'beam', 'series', 'h', 'lon', 'lat'], dtype='object')

In [62]:
df.series.unique()

array(['Noise', 'Buffer', 'Low', 'Medium', 'High'], dtype=object)

In [5]:
BATCH_SIZE = 15000 # fails after ~90k

In [6]:
len(df) // BATCH_SIZE

372

In [32]:
# df2 = df_high.iloc[:BATCH_SIZE,:]

In [84]:
# df2 = df.sample(n=BATCH_SIZE)

In [278]:
# extent = [8.46, 55.26, 8.63, 55.36]
extent = [8.46, 55.36, 8.62, 55.42]

df2 = df[(df.lon > extent[0]) & (df.lon < extent[2]) & (df.lat > extent[1]) & (df.lat < extent[3])]
df2 = df2[(df2.series != 'Noise')]
df2 = df2.copy()

In [279]:
len(df2)

87904

In [281]:
coords = list(zip(df2.lon, df2.lat))

In [282]:
g = ee.Geometry.MultiPoint(coords) # visualize using multi-point instead of feature collection - faster
pointsRGB = ee.FeatureCollection(g).style(pointSize=2, color='00000000', fillColor='ffeda0')
pointsRGB = pointsRGB.select(0).mask().focal_max(3).selfMask().visualize(palette=['000000'], opacity=0.5).blend(pointsRGB)

m = create_map()
m.addLayer(image.updateMask(image.gt(-0.09)), { 'min': -0.10, 'max': 0.15, 'palette': ['lightblue', 'black'] }, 'water occurrence')
m.addLayer(pointsRGB, {}, 'sampled points')
m.setCenter(8.5199, 55.3184, 11)
m.addLayerControl()
m

In [283]:
features = ee.List(coords).map(lambda o: ee.Feature(ee.Geometry.Point(coords=o)))
features = ee.FeatureCollection(features)
features.size().getInfo()

87904

In [284]:
values = image.reduceRegions(collection=features, reducer=ee.Reducer.first().setOutputs(['water_occurrence']), scale=10)
results = values.aggregate_array('water_occurrence').getInfo()

In [285]:
len(results)

87904

In [286]:
df2['water_occurrence'] = results

In [255]:
# plt.close('all')
fig, ax = plt.subplots(figsize=(10, 3))
hmin = 40
hmax = 42.5
df3 = df2[(df2.h > hmin) & (df2.h < hmax)]

df4 = df3[df3.series == 'High']
ax.plot(df4.water_occurrence, df4.h, 'k.', markersize=0.5, alpha=0.2)

df4 = df3[df3.series == 'Medium']
ax.plot(df4.water_occurrence, df4.h, 'r.', markersize=1)

df4 = df3[df3.series == 'Buffer']
ax.plot(df4.water_occurrence, df4.h, 'b.', markersize=1)

ax.set_xlabel('water occurrence')
ax.set_ylabel('elevation (ICESat-2)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [288]:
df2

,track_id,date,beam,series,h,lon,lat,water_occurrence
428,69,2019-01-01,gt3r,Buffer,50.817707,8.479082,55.377421,0.055138
429,69,2019-01-01,gt3r,Buffer,27.737589,8.478205,55.372444,0.036981
461,69,2019-01-01,gt3r,Low,36.980545,8.486457,55.419794,0.150000
462,69,2019-01-01,gt3r,Low,32.487840,8.484553,55.408769,0.150000
463,69,2019-01-01,gt3r,Low,50.159325,8.483429,55.402300,0.136790
...,...,...,...,...,...,...,...,...
5362765,1113,2019-12-08,gt1l,High,41.340310,8.486460,55.419846,0.150000
5362766,1113,2019-12-08,gt1l,High,41.397920,8.486443,55.419942,0.150000
5362767,1113,2019-12-08,gt1l,High,41.401497,8.486439,55.419961,0.150000
5362768,1113,2019-12-08,gt1l,High,41.593582,8.486438,55.419967,0.150000


In [292]:
df2.beam.unique()

array(['gt3r', 'gt3l', 'gt2r', 'gt2l', 'gt1r', 'gt1l'], dtype=object)

In [290]:
tracks = df2.track_id.unique()
tracks

array(['69', '671', '1113'], dtype=object)

In [293]:
for t in tracks:
    print(t, df[df.track_id == t].date.unique())

69 ['2019-01-01' '2019-04-02' '2019-10-01' '2019-12-31' '2020-03-31']
671 ['2018-11-11' '2019-02-10' '2019-05-12' '2019-08-11' '2019-11-09'
 '2020-02-08']
1113 ['2018-12-10' '2019-03-11' '2019-06-10' '2019-09-08' '2019-12-08'
 '2020-03-08']


In [291]:
# plt.close('all')
fig, ax = plt.subplots(figsize=(10, 3))
hmin = 39
hmax = 44
df3 = df2[(df2.h > hmin) & (df2.h < hmax)]


df4 = df3[(df3.series == 'High') & (df3.track_id == '69')]
ax.plot(df4.water_occurrence, df4.h, 'k.', markersize=0.5, alpha=0.2)

df4 = df3[(df3.series == 'High') & (df3.track_id == '671')]
ax.plot(df4.water_occurrence, df4.h, 'r.', markersize=0.5, alpha=0.2)

df4 = df3[(df3.series == 'High') & (df3.track_id == '1113')]
ax.plot(df4.water_occurrence, df4.h, 'b.', markersize=0.5, alpha=0.2)


# df4 = df3[df3.series == 'Medium']
# ax.plot(df4.water_occurrence, df4.h, 'r.', markersize=1)

# df4 = df3[df3.series == 'Buffer']
# ax.plot(df4.water_occurrence, df4.h, 'b.', markersize=1)

ax.set_xlabel('water occurrence')
ax.set_ylabel('elevation (ICESat-2)')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …